<a href="https://colab.research.google.com/github/yahyafaisal786/NLP-Assignment-CCP-ChatBot-for-Travel-and-Hospitability-/blob/main/NLP_Assignment_CCP_(ChatBot_for_Travel_and_Hospitability).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import ne_chunk, pos_tag
from textblob import TextBlob
import requests

# Uncomment these lines if running for the first time to download necessary NLTK data
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

services = {
    "hotel": {
        "name": "Luxury Beachfront Hotel",
        "price": "$250 per night",
        "description": "A 5-star hotel with stunning ocean views and world-class amenities."
    },
    "flight": {
        "name": "Economy Class Flight",
        "price": "$499 round trip",
        "description": "Comfortable economy class seating with in-flight entertainment."
    },
    "car_rental": {
        "name": "Premium Car Rental",
        "price": "$70 per day",
        "description": "Luxury car rental service with a wide range of premium vehicles."
    },
    "tour_package": {
        "name": "All-Inclusive Tour Package",
        "price": "$1500",
        "description": "A week-long tour package including flights, hotels, and guided tours."
    }
}

def preprocess_input(user_input):
    tokens = word_tokenize(user_input.lower())
    filter_tokens = [word for word in tokens if word not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filter_tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return stemmed_tokens

def search_service(user_input):
    keywords = preprocess_input(user_input)
    match_services = []
    for service_key, service_info in services.items():
        name_tokens = preprocess_input(service_info['name'])
        if any(keyword in name_tokens for keyword in keywords):
            match_services.append(service_info)
    return match_services

def ner(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    named_entities = ne_chunk(tagged_words)
    ner_dict = {}
    for entity in named_entities:
        if isinstance(entity, nltk.tree.Tree):
            entity_type = entity.label()
            entity_name = " ".join([word for word, tag in entity.leaves()])
            ner_dict[entity_name] = entity_type
    return ner_dict

def get_weather(city):
    api_key = "f75cce498b102285c63b44cff481c682"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()
    if data["cod"] == "404":
        return "Sorry, city not found."
    main = data["main"]
    weather = data["weather"][0]["description"]
    return f"The current weather in {city} is {weather} with a temperature of {main['temp']}°C."

greetings = ["hello", "hi", "hey"]
exit_cmds = ["bye", "goodbye", "exit", "thanks", "thank you"]
booking_issues = ["book", "booking", "reservation", "reserve"]

def respond(user_input):
    response = ""
    blob = TextBlob(user_input)
    sentiment = blob.sentiment.polarity

    if sentiment > 0.5:
        response += "I'm glad you're excited! "
    elif sentiment < -0.5:
        response += "I'm sorry you're having issues. "

    tag_words = pos_tag(word_tokenize(user_input))
    nouns = [word for word, pos in tag_words if pos.startswith('NN')]
    potential_services = []

    ent = ner(user_input)
    for entity, entity_type in ent.items():
        if entity_type == 'GPE':
            matching_services = search_service(entity)
            if matching_services:
                potential_services.extend(matching_services)

    if user_input.lower() in greetings:
        return response + "Hello! How can I assist you with your travel plans today?"
    elif user_input.lower() in exit_cmds:
        return response + "You're welcome! Have a great trip!"
    elif any(issue in user_input.lower() for issue in booking_issues):
        return response + "You can make reservations through our website or by calling our customer support. If you have any issues, please contact support@travelagency.com."

    if "weather" in user_input.lower():
        city = [word for word, pos in tag_words if pos == 'NNP'][0]
        weather_info = get_weather(city)
        return response + weather_info

    if nouns or potential_services:
        if nouns:
            for noun in nouns:
                matching_services = search_service(noun)
                if matching_services:
                    potential_services.extend(matching_services)

    if potential_services:
        for service_info in potential_services:
            response += f"We have the following service available: {service_info['name']}, Price - {service_info['price']}, Description - {service_info['description']}\n"
    else:
        return response + "I'm sorry, I couldn't find any matching services."

    return response

print("Bot: Hello! How can I assist you with your travel plans today?")

while True:
    user_input = input("You: ")
    bot_response = respond(user_input)
    print("Bot:", bot_response)

    if user_input.lower() in exit_cmds:
        break


Bot: Hello! How can I assist you with your travel plans today?
You: Hello
Bot: Hello! How can I assist you with your travel plans today?
You: I'm looking for a hotel in Istanbul
Bot: We have the following service available: Luxury Beachfront Hotel, Price - $250 per night, Description - A 5-star hotel with stunning ocean views and world-class amenities.

You: I am looking for a complete tour pakage
Bot: We have the following service available: All-Inclusive Tour Package, Price - $1500, Description - A week-long tour package including flights, hotels, and guided tours.

You: What's the weather in Istanbul?
Bot: The current weather in Istanbul is clear sky with a temperature of 31.12°C.
You: Thank you
Bot: You're welcome! Have a great trip!
